In [72]:
import pickle
import pandas as pd
import numpy as np
import operator
from num2words import num2words
from sklearn.metrics import accuracy_score

In [2]:
with open('../../output/base_dict.dump', 'rb') as f:
    res = pickle.load(f)

In [3]:
test = pd.read_csv('../../input/en_test.csv')

In [4]:
test.shape

(1088564, 3)

In [5]:
test.head()

,sentence_id,token_id,before
0,0,0,Another
1,0,1,religious
2,0,2,family
3,0,3,is
4,0,4,of


In [6]:
if res.get(test["before"][0], None):
    print("a")

a


In [7]:
"a" in res

True

In [8]:
n_OOV = 0
OOVlist = []
def count_oov(x):
    global n_OOV, OOVlist, res
    if not x in res:
        if not x in OOVlist:
            OOVlist.append(x)
        n_OOV += 1
    return None
test["before"].apply(lambda x: count_oov(x))
print()

In [9]:
test["before"].nunique()

116854

In [10]:
test.shape

(1088564, 3)

In [11]:
n_OOV

78468

In [12]:
len(OOVlist)

6634

In [13]:
OOVlist

[',',
 '"',
 '52.63 km2',
 'Ghallu',
 'Rajhistan',
 '11.8 MB',
 'http://www.britannica.com/EBchecked/topic/285248/India',
 'IndiaEncyclopedia',
 'http://search.eb.com/eb/article-9061377',
 'Gtkmm',
 'SANAM',
 "Supastars'",
 'Cechin',
 'Charrin',
 'Plubbers',
 '$121,141',
 "Ahsoka's",
 'puuyi',
 'pathanu',
 'gulka',
 'ayimka',
 'theshots.co.uk',
 'Peuralinna',
 'Kaldova',
 'barid',
 'Reterink',
 'Desseria',
 'colation',
 'Yountsville',
 '526.1/km2',
 'Prescr',
 'PopServices',
 'supplementationAnti',
 'ghin',
 'Broadlink',
 'Devacarya',
 'Svabhurama',
 'automechanics',
 'Benosa',
 'Cynlais',
 '277.61 km2',
 'Talicud',
 'Estberg',
 '17th December 2015',
 'BorophaginaeX',
 'AssisiPehowski',
 'Kovachki',
 'Anagé',
 '09/08/1943',
 'CarbinesAyra',
 '1,237.1',
 '477.6/km2',
 'IIAustraliaCanadaCzechoslovakia',
 'Lozane',
 'apni',
 'rahna',
 'fitrat',
 'usool',
 'oargi',
 'hersir',
 'Naumudali',
 'Hallbjarnar',
 'halftrolls',
 'Ketils',
 'haeings',
 'unnumberedFerraro',
 '2.5 million USD',
 '045

## 課題の切り分け
1. OOVの扱い方(78,468/1,088,564個)
  * ルールベース
  * Trainで類似性の高い単語を参考にする
1. NonOOVの候補の選択方法
  * RNNからのSoftmax
  

## validデータで精度検証

In [14]:
train = pd.read_csv('../../input/en_train.csv')

In [15]:
train.shape

(9918441, 5)

In [16]:
train.head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of


In [17]:
train["sentence_id"].nunique()

748066

In [18]:
valid = train[train["sentence_id"]>=670000]

In [19]:
valid.shape

(1053392, 5)

In [20]:
sdict = {}
sdict['km2'] = 'square kilometers'
sdict['km'] = 'kilometers'
sdict['kg'] = 'kilograms'
sdict['lb'] = 'pounds'
sdict['dr'] = 'doctor'
sdict['m²'] = 'square meters'

In [23]:
def oleg(line):
    line = str(line)
    if line in res:
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        return srtd[0][0]
    else:
        # line.split(' ')
        if len(line) > 1:
            #カンマ区切りの数字を数字のみに変換
            val = line.split(',')
            if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                line = ''.join(val)

        if line.isdigit():
            #数字を変換
            srtd = line.translate(SUB)
            srtd = srtd.translate(SUP)
            srtd = srtd.translate(OTH)
            return num2words(float(srtd))
        elif len(line.split(' ')) > 1:
            #数字+単位を変換
            val = line.split(' ')
            for i, v in enumerate(val): 
                if v.isdigit():
                    srtd = v.translate(SUB)
                    srtd = srtd.translate(SUP)
                    srtd = srtd.translate(OTH)
                    val[i] = num2words(float(srtd))
                elif v in sdict:
                    val[i] = sdict[v]
            return ' '.join(val)
        else:
            return line

In [24]:
valid["pred"] = valid["before"].apply(lambda x: oleg(x))

/home/shinoda/app/pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
valid.head()

,sentence_id,token_id,class,before,after,pred
8865049,670000,0,PLAIN,Members,Members,Members
8865050,670000,1,PLAIN,of,of,of
8865051,670000,2,PLAIN,the,the,the
8865052,670000,3,PLAIN,mob,mob,mob
8865053,670000,4,PLAIN,escorted,escorted,escorted


In [30]:
after=valid["after"].values
pred=valid["pred"].values

In [33]:
np.sum(after==pred)/valid.shape[0]

0.99765519388793533

In [36]:
pd.set_option('display.max_rows', 100)

In [37]:
valid[after!=pred]

,sentence_id,token_id,class,before,after,pred
8865415,670024,11,PLAIN,-,to,-
8865461,670027,9,PLAIN,-,to,-
8866954,670133,6,PLAIN,-,to,-
8867307,670158,24,PLAIN,-,to,-
8867870,670194,3,PLAIN,:,to,:
8868136,670212,15,ORDINAL,V,fifth,V
8868656,670250,13,ORDINAL,I,the first,I
8869648,670326,15,CARDINAL,1947,one thousand nine hundred forty seven,nineteen forty seven
8871253,670439,1,PLAIN,-,to,-
8871272,670439,20,PLAIN,-,to,-


In [49]:
len(res)

4046878

In [57]:
a = {"a":1, "b":2}

In [61]:
a.values()

dict_values

In [59]:
np.sum(list(map(lambda x: len(x)>1, res.values())))

14413

In [62]:
np.sum(list(map(lambda x: len(x), res.values())))

4062394

In [64]:
(4062394-4046878 + 14413)/14413

2.0765281343231803

In [69]:
train[train["before"]=="-"].shape

(22635, 5)

In [70]:
test[test["before"]=="-"].shape

(4186, 3)

In [71]:
test.shape

(1088564, 3)

In [46]:
train[train["sentence_id"]==14]

,sentence_id,token_id,class,before,after
157,14,0,PUNCT,"""",""""
158,14,1,DATE,1987,nineteen eighty seven
159,14,2,LETTERS,ALCS,a l c s
160,14,3,PLAIN,Game,Game
161,14,4,CARDINAL,2,two
162,14,5,VERBATIM,-,-
163,14,6,PLAIN,Detroit,Detroit
164,14,7,PLAIN,Tigers,Tigers
165,14,8,PLAIN,vs,versus


In [47]:
train[train["sentence_id"]==44]

,sentence_id,token_id,class,before,after
550,44,0,PLAIN,Thus,Thus
551,44,1,PUNCT,",",","
552,44,2,PLAIN,during,during
553,44,3,PLAIN,the,the
554,44,4,PLAIN,years,years
555,44,5,DATE,1573,fifteen seventy three
556,44,6,PLAIN,-,to
557,44,7,DATE,1595,fifteen ninety five
558,44,8,PLAIN,they,they
559,44,9,PLAIN,rebuilt,rebuilt


In [65]:
valid[valid["class"]=="DATE"]

,sentence_id,token_id,class,before,after,pred
8865068,670001,7,DATE,1987,nineteen eighty seven,nineteen eighty seven
8865242,670012,9,DATE,1910,nineteen ten,nineteen ten
8865374,670021,3,DATE,2002,two thousand two,two thousand two
8865385,670023,1,DATE,1869,eighteen sixty nine,eighteen sixty nine
8865402,670023,18,DATE,1876,eighteen seventy six,eighteen seventy six
8865457,670027,5,DATE,2010,twenty ten,twenty ten
8865460,670027,8,DATE,2011,twenty eleven,twenty eleven
8865472,670028,4,DATE,2011,twenty eleven,twenty eleven
8865504,670030,2,DATE,December 2015,december twenty fifteen,december twenty fifteen
8865563,670033,16,DATE,"August 14, 2001",august fourteenth two thousand one,august fourteenth two thousand one


In [85]:
res["<TWO_DIGIT>"]

KeyError: '<TWO_DIGIT>'